In [1]:
import os
import json
import pickle
import torch
import numpy as np
from learned_simulator import Simulator

INPUT_SEQUENCE_LENGTH = 6
noise_std = 6.7e-4
with open('data/WaterRamps/metadata.json', 'rt') as f:
    metadata = json.loads(f.read())

num_steps = metadata['sequence_length'] - INPUT_SEQUENCE_LENGTH
normalization_stats = {
        'acceleration': {
            'mean':torch.FloatTensor(metadata['acc_mean']).to('cuda'), 
            'std':torch.sqrt(torch.FloatTensor(metadata['acc_std'])**2 + noise_std**2).to(),
        }, 
        'velocity': {
            'mean':torch.FloatTensor(metadata['vel_mean']).to('cuda'), 
            'std':torch.sqrt(torch.FloatTensor(metadata['vel_std'])**2 + noise_std**2).to('cuda'),
        }, 
    }


/local/rmenezes/miniconda3/envs/graph_simulation/lib/python3.9/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: 
attribute assignment is not defined on __torch__.torch_sparse.storage.SparseStorage:
  File "/local/rmenezes/miniconda3/envs/graph_simulation/lib/python3.9/site-packages/torch_sparse/storage.py", line 145
    def empty(self):
        self = SparseStorage.__new__(SparseStorage)
        self._row = None
        ~~~~~~~~~~~~~~~~ <--- HERE
        self._rowptr = None
        self._value = None

  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
simulator = Simulator(
        particle_dimension=2,
        node_in=30,
        edge_in=3,
        latent_dim=128,
        num_message_passing_steps=12,
        mlp_num_layers=2,
        mlp_hidden_dim=128,
        connectivity_radius=metadata['default_connectivity_radius'],
        boundaries=np.array(metadata['bounds']),
        normalization_stats=normalization_stats,
        num_particle_types=9,
        particle_type_embedding_size=16,
        device='cuda',
    )

simulator.load('train_log/run26/model.pth')
simulator.cuda()


Simulator(
  (_particle_type_embedding): Embedding(9, 16)
  (_encode_process_decode): EncodeProcessDecode(
    (_encoder): Encoder(
      (node_fn): Sequential(
        (0): Sequential(
          (0): Linear(in_features=30, out_features=128, bias=True)
          (1): ReLU()
          (2): Linear(in_features=128, out_features=128, bias=True)
          (3): ReLU()
          (4): Linear(in_features=128, out_features=128, bias=True)
          (5): Identity()
        )
        (1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (edge_fn): Sequential(
        (0): Sequential(
          (0): Linear(in_features=3, out_features=128, bias=True)
          (1): ReLU()
          (2): Linear(in_features=128, out_features=128, bias=True)
          (3): ReLU()
          (4): Linear(in_features=128, out_features=128, bias=True)
          (5): Identity()
        )
        (1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
    )
    (_processor): Processor()
    (_dec

In [5]:
import os
import json
import pickle
import torch

from tf_data_reader import prepare_data_from_tfds

INPUT_SEQUENCE_LENGTH = 6

with open('data/metadata.json', 'rt') as f:
    metadata = json.loads(f.read())

num_steps = metadata['sequence_length'] - INPUT_SEQUENCE_LENGTH


def infer(simulator, data_path="data/valid.tfrecord", device="cuda"):
    ds = prepare_data_from_tfds(data_path=data_path, is_rollout=True)
    eval_rollout(ds, simulator, num_steps=num_steps, save_results=True, device=device)


def eval_single_rollout(simulator, features, num_steps, device):
    initial_positions = features['position'][:, 0:INPUT_SEQUENCE_LENGTH]
    ground_truth_positions = features['position'][:, INPUT_SEQUENCE_LENGTH:]
    
    current_positions = initial_positions
    predictions = []
    for step in range(num_steps):
        next_position = simulator.predict_positions(
            current_positions,
            n_particles_per_example=features['n_particles_per_example'],
            particle_types=features['particle_type'],
        ) # (n_nodes, 2)
        # Update kinematic particles from prescribed trajectory.
        kinematic_mask = (features['particle_type'] == 3).clone().detach().to(device)
        next_position_ground_truth = ground_truth_positions[:, step]
        kinematic_mask = kinematic_mask.bool()[:, None].expand(-1, 2)
        next_position = torch.where(kinematic_mask, next_position_ground_truth, next_position)
        predictions.append(next_position)
        current_positions = torch.cat([current_positions[:, 1:], next_position[:, None, :]], dim=1)
    predictions = torch.stack(predictions) # (time, n_nodes, 2)
    ground_truth_positions = ground_truth_positions.permute(1,0,2)
    loss = (predictions - ground_truth_positions) ** 2
    output_dict = {
        'initial_positions': initial_positions.permute(1,0,2).cpu().numpy(),
        'predicted_rollout': predictions.cpu().numpy(),
        'ground_truth_rollout': ground_truth_positions.cpu().numpy(),
        'particle_types': features['particle_type'].cpu().numpy(),
    }
    return output_dict, loss


def eval_rollout(ds, simulator, num_steps, num_eval_steps=1, save_results=False, device='cuda'):
    eval_loss = []
    i = 0
    simulator.eval()
    with torch.no_grad():
        for example_i, (features, labels) in enumerate(ds):
            features['position'] = torch.tensor(features['position']).to(device) # (n_nodes, 600, 2)
            features['n_particles_per_example'] = torch.tensor(features['n_particles_per_example']).to(device)
            features['particle_type'] = torch.tensor(features['particle_type']).to(device)
            labels = torch.tensor(labels).to(device)
            example_rollout, loss = eval_single_rollout(simulator, features, num_steps, device)
            example_rollout['metadata'] = metadata
            eval_loss.append(loss)
            if save_results: 
                example_rollout['metadata'] = metadata
                filename = f'rollout_{example_i}.pkl'
                filename = os.path.join('rollouts/', filename)
                with open(filename, 'wb') as f:
                    pickle.dump(example_rollout, f)
            i += 1
            print(f'{i} / {num_eval_steps}')
            if i >= num_eval_steps:
                break
    simulator.train()
    return torch.stack(eval_loss).mean(0)


2024-07-10 19:30:04.430875: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 19:30:05.087149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/local/rmenezes/miniconda3/envs/graph_simulation/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
infer(
    simulator, 
    data_path='data/WaterRamps/valid.tfrecord',
    device='cuda'
)

2024-07-10 19:30:05.723147: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 19:30:05.724123: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!